### Today, we will scrape Producthunt with Selenium

Selenium is a tool initially created to automate tests on websites. It is therefore very useful when information is accessible by clicking on links. A button for example is an element on which it is very difficult to obtain the link. Beautifull soup then becomes limited.
In this case, use Selenium

### Load libraries

In [1]:
import bs4
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import json
import re
import lxml.html
import time
import random
from random import randint
import logging
import collections
from time import gmtime, strftime

import re
from tabulate import tabulate
import time
import os
date=strftime("%Y-%m-%d")


### Install Selenium according to this manual

https://selenium-python.readthedocs.io/installation.html#downloading-python-bindings-for-selenium/bin

nb: Linux: put your geckodriver (the downloaded extension) in the equivalent path at home to /home/YOURNAME/.local/bin

We will simulate a search on the official Phython website

In [3]:
import selenium

# The selenium.webdriver module provides all the implementations of WebDriver. WebDriver implementations 
# Currently supported are Firefox, Chrome, IE and Remote. The Keys class provides keys in 
# the keyboard such as RETURN, F1, ALT etc.
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# Then, the instance of Firefox WebDriver is created.
driver = webdriver.Chrome()
# The driver.get method will lead to a page given by the URL. WebDriver will wait until the page is fully completed 
# loaded (i.e. the "onload" event has been triggered) before returning the control to your script. 
# It should be noted that if your page uses a lot of AJAX when loading, WebDriver may not know 
# when it was fully charged:
driver.get("http://www.python.org")
# The following line is a statement confirming that the title contains the word "Python"
assert "Python" in driver.title
# WebDriver offers several methods to search for items using one of the methods 
# find_element_by_by_ * . For example, the input text element can be located by its name attribute by 
# using the find_element_by_name method 
elem = driver.find_element_by_name("q")
# Then we send keys. This is similar to entering keys using your keyboard. 
# Special keys can be sent using the imported selenium.webdriver.common.keys Keys class. 
# For security reasons, we will delete any pre-filled text in the input field 
# (for example, "Search") so that it does not affect our search results:
# Han Added comment

elem.clear()
elem.send_keys("pycon")
elem.send_keys(Keys.RETURN)
time.sleep(5)
#After submitting the page, you should get the result if there is one. To ensure that certain results 
# are found, make an assertion:
assert "No results found." not in driver.page_source


driver.close()

WebDriverException: Message: chrome not reachable
  (Session info: chrome=90.0.4430.93)


#### Open the source code of this page (HTML) and check that the search area (field) is called "q".

input id="id-search-field" name="q" type="search" role="textbox" class="search-field placeholder" placeholder="Search" value="" tabindex="1"

### Getting a phone number from an ad in the right place

In [5]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By


url='https://www.leboncoin.fr/livres/1963802714.htm?ac=1481102916'
print('ready')
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)
time.sleep(15) ##there is some captcha to fill, I leave some time to complete them manually
elem = driver.find_element_by_xpath("//button[@title='voir le numéro']")
time.sleep(5)
elem.click()
time.sleep(5)
soup=BeautifulSoup(driver.page_source)
driver.close()
print(soup.find('a', attrs={'class':'_2qvLx _3osY2 _35pAC _1Vw3w _kC3e _32ILh _2L9kx _30q3D _1y_ge _3QJkO'}).text)

ready


WebDriverException: Message: chrome not reachable
  (Session info: chrome=90.0.4430.93)


### Starting from an ad in the right place, collect all the information available to define the product being sold. Use selenium for the telephone number

In [20]:
url ="https://www.leboncoin.fr/livres/1963802714.htm?ac=1481102916"
r = requests.get(url)
print(url, r.status_code)

soup = BeautifulSoup(r.content,'lxml')
print(soup.content)
with open('source_code_le_bon_coin.txt', 'w') as source:
    source.write(r.text)
##there is a scrapper detector and I don't have access to the source code webpage

    

https://www.leboncoin.fr/livres/1963802714.htm?ac=1481102916 403
None


In [58]:
url='https://www.leboncoin.fr/livres/1963802714.htm?ac=1481102916'
print('ready')
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)
time.sleep(15)
elem = driver.find_element_by_xpath("//button[@title='voir le numéro']")
time.sleep(5)
elem.click()
time.sleep(5)
soup=BeautifulSoup(driver.page_source)
driver.close()

product_info ={
    'name': soup.find('h1').text,
    'price' : soup.find('div', attrs={'data-qa-id':"adview_price"}).text,
    'state' : soup.find('span', attrs={'class':"_3eNLO _38n__ _137P- P4PEa _35DXM"}).text,
    'description' : soup.find('div', attrs={'data-qa-id':"adview_description_container"}).text,
    'place' : soup.find_all('a', attrs={'class':"_64Mha _2NG-q _1GcfX Dqdzf cJtdT _3j0OU"})[4].text,
    'phone number' : soup.find('a', attrs={'class':'_2qvLx _3osY2 _35pAC _1Vw3w _kC3e _32ILh _2L9kx _30q3D _1y_ge _3QJkO'}).text

}


for key, value in product_info.items():
    print(key, ": ", value)

ready
name :  Manuel Finances Publiques Droit
price :  15 €
state :  État neuf
description :  Bonjour,

Je vends ce manuel neuf à 15€, acheté 19.50€ à la Fnac en début d’année. Il n’a jamais servi et est en très bon état. 

Dépôt en main propre possible sur Rennes ou sur Chartres de Bretagne. Contact uniquement par SMS.

Cordialement
place :  Rennes 35000
phone number :  06 32 72 04 58


### API (Application Program Interface)

A set of tools and methods that allow different applications to interact with each other. In the case of a web service, we can retrieve data dynamically. By using an API correctly, we can thus obtain in real time, the modifications made on a "parent" site.

For example, we will retrieve online news, for example from the "L'équipe" website.

Follow the instructions at https://newsapi.org/s/lequipe-api to retrieve an "API key" connection key

Your API key is: 73bbb95f8ecb49b499113a46481b4af1


It is frequent that a key does not work after a while 5mins 30 mins 1 d ...
So don't jump up if you get an error message back.

In [9]:
import requests
key='55f52879ce2742758eec9a4bbfc31cdd'
url ='https://newsapi.org/v2/top-headlines?sources=lequipe&apiKey='+key
response = requests.get(url)

# Here the response format is a json file, it is used as a dictionary

print (response.json())

{'status': 'ok', 'totalResults': 10, 'articles': [{'source': {'id': 'lequipe', 'name': "L'equipe"}, 'author': "L'EQUIPE", 'title': 'Miles Scotson en solitaire pour la 1re étape du Tour de Valence', 'description': "Miles Scotson (Groupama-FDJ) s'est échappé à vingt kilomètres de l'arrivée.", 'url': 'https://www.lequipe.fr/Cyclisme-sur-route/Actualites/Miles-scotson-en-solitaire-pour-la-1re-etape-du-tour-de-valence/1242586', 'urlToImage': 'https://medias.lequipe.fr/img-photo-jpg/miles-scotson-victorieux-a-ondara-luis-angel-gomez-bettiniphoto-presse-sports/1500000001475644/0:0,1998:1332-640-427-75/2d7de.jpg', 'publishedAt': '2021-04-14T14:10:00+00:00', 'content': "La 1re étape du Tour de la Communauté de Valence a été remportée mercredi par l'Australien Miles Scotson (Groupama-FDJ), après être sorti du peloton à une vingtaine de kilomètres de l'arrivée. Sous l… [+162 chars]"}, {'source': {'id': 'lequipe', 'name': "L'equipe"}, 'author': "L'EQUIPE", 'title': 'Des primes revalorisées pour le

In [10]:
dic=response.json()
print(dic.keys())

dict_keys(['status', 'totalResults', 'articles'])


In [11]:
for elem in list(dic.keys()):
    print('##############################################')
    print("clé: ",elem,"// values: ", dic[elem])

##############################################
clé:  status // values:  ok
##############################################
clé:  totalResults // values:  10
##############################################
clé:  articles // values:  [{'source': {'id': 'lequipe', 'name': "L'equipe"}, 'author': "L'EQUIPE", 'title': 'Miles Scotson en solitaire pour la 1re étape du Tour de Valence', 'description': "Miles Scotson (Groupama-FDJ) s'est échappé à vingt kilomètres de l'arrivée.", 'url': 'https://www.lequipe.fr/Cyclisme-sur-route/Actualites/Miles-scotson-en-solitaire-pour-la-1re-etape-du-tour-de-valence/1242586', 'urlToImage': 'https://medias.lequipe.fr/img-photo-jpg/miles-scotson-victorieux-a-ondara-luis-angel-gomez-bettiniphoto-presse-sports/1500000001475644/0:0,1998:1332-640-427-75/2d7de.jpg', 'publishedAt': '2021-04-14T14:10:00+00:00', 'content': "La 1re étape du Tour de la Communauté de Valence a été remportée mercredi par l'Australien Miles Scotson (Groupama-FDJ), après être sorti du peloton 

In [12]:
# And now we have lists in dictionaries it's json but it's always the same thing
# We will discover the information of the article key
for elem in enumerate(dic['articles']):
    print('###############################################')
    print(elem)

###############################################
(0, {'source': {'id': 'lequipe', 'name': "L'equipe"}, 'author': "L'EQUIPE", 'title': 'Miles Scotson en solitaire pour la 1re étape du Tour de Valence', 'description': "Miles Scotson (Groupama-FDJ) s'est échappé à vingt kilomètres de l'arrivée.", 'url': 'https://www.lequipe.fr/Cyclisme-sur-route/Actualites/Miles-scotson-en-solitaire-pour-la-1re-etape-du-tour-de-valence/1242586', 'urlToImage': 'https://medias.lequipe.fr/img-photo-jpg/miles-scotson-victorieux-a-ondara-luis-angel-gomez-bettiniphoto-presse-sports/1500000001475644/0:0,1998:1332-640-427-75/2d7de.jpg', 'publishedAt': '2021-04-14T14:10:00+00:00', 'content': "La 1re étape du Tour de la Communauté de Valence a été remportée mercredi par l'Australien Miles Scotson (Groupama-FDJ), après être sorti du peloton à une vingtaine de kilomètres de l'arrivée. Sous l… [+162 chars]"})
###############################################
(1, {'source': {'id': 'lequipe', 'name': "L'equipe"}, 'author':

In [13]:
# So if we keep going, it gives us another dictionary!
for elem in dic['articles'][0].keys():
    print(' Key : ',elem,'Values : ',dic['articles'][0][elem])

 Key :  source Values :  {'id': 'lequipe', 'name': "L'equipe"}
 Key :  author Values :  L'EQUIPE
 Key :  title Values :  Miles Scotson en solitaire pour la 1re étape du Tour de Valence
 Key :  description Values :  Miles Scotson (Groupama-FDJ) s'est échappé à vingt kilomètres de l'arrivée.
 Key :  url Values :  https://www.lequipe.fr/Cyclisme-sur-route/Actualites/Miles-scotson-en-solitaire-pour-la-1re-etape-du-tour-de-valence/1242586
 Key :  urlToImage Values :  https://medias.lequipe.fr/img-photo-jpg/miles-scotson-victorieux-a-ondara-luis-angel-gomez-bettiniphoto-presse-sports/1500000001475644/0:0,1998:1332-640-427-75/2d7de.jpg
 Key :  publishedAt Values :  2021-04-14T14:10:00+00:00
 Key :  content Values :  La 1re étape du Tour de la Communauté de Valence a été remportée mercredi par l'Australien Miles Scotson (Groupama-FDJ), après être sorti du peloton à une vingtaine de kilomètres de l'arrivée. Sous l… [+162 chars]


### Make a script that allows you to take details of the last ten news from the team or another site. Store them in a nice csv or excel file

In [18]:
title = []
author = []
content = []
for i in range(10):
    title.append(dic['articles'][i]['title'])
    author.append(dic['articles'][i]['author'])
    content.append(dic['articles'][i]['content'])
print(author)

["L'EQUIPE", "L'EQUIPE", "L'EQUIPE", "L'EQUIPE", "L'EQUIPE", "L'EQUIPE", "L'EQUIPE", "L'EQUIPE", "L'EQUIPE", "L'EQUIPE"]


In [19]:
import pandas as pd
df=pd.DataFrame({'Titre':title})
df['auteur']=author
df['article']=content

In [20]:
df.to_csv('./assets/articles.csv', index=False)

In [5]:
driver = webdriver.Chrome()
# The driver.get method will lead to a page given by the URL. WebDriver will wait until the page is fully completed 
# loaded (i.e. the "onload" event has been triggered) before returning the control to your script. 
# It should be noted that if your page uses a lot of AJAX when loading, WebDriver may not know 
# when it was fully charged:
driver.get("https://immo.vlan.be/fr")
# The following line is a statement confirming that the title contains the word "Python"
assert "vlan" in driver.title
# WebDriver offers several methods to search for items using one of the methods 
# find_element_by_by_ * . For example, the input text element can be located by its name attribute by 
# using the find_element_by_name method 

time.sleep(15)
elem = driver.find_element_by_xpath("//button[@title='voir le numéro']")
# Then we send keys. This is similar to entering keys using your keyboard. 
# Special keys can be sent using the imported selenium.webdriver.common.keys Keys class. 
# For security reasons, we will delete any pre-filled text in the input field 
# (for example, "Search") so that it does not affect our search results:


elem.clear()
elem.send_keys("7000")
elem.send_keys(Keys.RETURN)
time.sleep(15)
#After submitting the page, you should get the result if there is one. To ensure that certain results 
# are found, make an assertion:
assert "No results found." not in driver.page_source


driver.close()

AssertionError: 

In [44]:

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains 
url='https://immo.vlan.be/fr'
print('ready')
source_pages = []
source_pages_url = []
driver = webdriver.Chrome()
driver.implicitly_wait(10)
driver.get(url)
time.sleep(7)
elem = driver.find_element_by_xpath("//input[@placeholder='Où ? Ville, Code Postal, Province ou Région.']")
elem.clear()
elem.send_keys("7000")
elem.send_keys(Keys.RETURN)
time.sleep(7)
source_pages.append(driver.page_source)
source_pages_url.append(driver.current_url)

elem = driver.find_element_by_xpath("//a[@title='suivant']")
actions = ActionChains(driver)
actions.move_to_element(elem)
actions.click(elem)
actions.perform()
time.sleep(7)
source_pages.append(driver.page_source)
source_pages_url.append(driver.current_url)

driver.close()
#print(soup)
#my_url = soup.find('a', attrs={'class':"détails"}).text

soup=BeautifulSoup(source_pages[0])
for link in soup.find_all('a'):
    detail_url = link.get('href')
    if detail_url == None:
        continue
    elif 'detail' in detail_url:
        print(detail_url)

soup=BeautifulSoup(source_pages[1])
for link in soup.find_all('a'):
    detail_url = link.get('href')
    if detail_url == None:
        continue
    elif 'detail' in detail_url:
        print(detail_url)
print(len(source_pages))
print(source_pages_url)

#product_info ={
    #'url': soup.find('a'attrs={'class':"détails"}).href,
    #'type' : soup.find('div', attrs={'data-qa-id':"adview_price"}).text,
    #'code postal' : 7000,

#} /html/body/div[2]/div[4]/div[2]/div/div[1]/div/div/div/div/div[2]/div[2]/div/section/div[4]/div/div/div/ul/li[10]/a

ready
/fr/detail/terrain-a-batir/a-vendre/6180/courcelles/var59104
/fr/detail/appartement/a-vendre/7170/la-hestre/var58606
/fr/detail/maison/a-vendre/5190/ham-sur-sambre/var57536
/fr/detail/maison/a-vendre/4400/flemalle/var57146
/fr/detail/immeuble-mixte/a-vendre/7120/peissant/var57020
/fr/detail/appartement/a-vendre/2260/tongerlo/raw65225
/fr/detail/maison/a-vendre/2620/hemiksem/raw65031
/fr/detail/maison/a-vendre/2990/wuustwezel/raw63959
/fr/detail/villa/a-vendre/3080/tervuren/raw62354
/fr/detail/maison/a-vendre/8530/harelbeke/raw62256
/fr/detail/maison-de-maitre/a-vendre/9300/alost/raw61933
/fr/detail/maison/a-vendre/8610/handzame/raw60048
/fr/detail/maison/a-vendre/8680/koekelare/raw60045
/fr/detail/maison/a-vendre/8501/bissegem/raw60036
/fr/detail/appartement/a-vendre/8890/dadizele/raw60035
/fr/detail/maison/a-vendre/9700/audenarde/raw59956
/fr/detail/appartement/a-vendre/9600/renaix/raw59914
/fr/detail/maison/a-vendre/8630/houtem/raw59658
/fr/detail/maison/a-vendre/2310/rijkevors

In [35]:
import requests
next_url =new_url[0:-1]+"2"
print(new_url)
r = requests.get(next_url)
print(next_url, r.status_code)
soup = BeautifulSoup(r.content)
print(r.content)

for link in soup.find_all('a'):
    detail_url = link.get('href')
    if detail_url == None:
        continue
    elif 'detail' in detail_url:
        print(detail_url)


https://immo.vlan.be/fr/immobilier?transactiontypes=a-vendre,en-vente-publique&noindex=1
https://immo.vlan.be/fr/immobilier?transactiontypes=a-vendre,en-vente-publique&noindex=2 200
b'<!doctype html>\n<html lang="en">\n<head>\n    <meta charset="utf-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n    <meta name="theme-color" content="#4e2780">\n    \n    <link rel="icon" type="image/ico" href="http://immo.vlan.be/Content/Images/favicon.ico">\n    <title>ImmoVlan.be</title>\n    <meta name="robots" content="noindex">\n\n\t<link href="http://immo.vlan.be/Content/css/bootstrap-md/bootstrap.min.css" rel="stylesheet"/>\n\t<link href="http://immo.vlan.be/Content/css/bootstrap-md/mdb.min.css" rel="stylesheet"/>\n\t<link href="http://immo.vlan.be/Content/css/bootstrap-override.css" rel="stylesheet"/>\n</head>\n\n<body class="detail-page not-home en-lang" style="overflow: visible; padding-right: 0 !importa

In [23]:
print(new_url[0:-1]+"2")

https://immo.vlan.be/fr/immobilier?transactiontypes=a-vendre,en-vente-publique&noindex=2


In [43]:
source_pages_url
soup=BeautifulSoup(source_pages[1])
for link in soup.find_all('a'):
    detail_url = link.get('href')
    if detail_url == None:
        continue
    elif 'detail' in detail_url:
        print(detail_url)

/fr/detail/terrain-a-batir/a-vendre/6180/courcelles/var59104
/fr/detail/appartement/a-vendre/7170/la-hestre/var58606
/fr/detail/maison/a-vendre/5190/ham-sur-sambre/var57536
/fr/detail/maison/a-vendre/4400/flemalle/var57146
/fr/detail/immeuble-mixte/a-vendre/7120/peissant/var57020
/fr/detail/appartement/a-vendre/2260/tongerlo/raw65225
/fr/detail/maison/a-vendre/2620/hemiksem/raw65031
/fr/detail/maison/a-vendre/2990/wuustwezel/raw63959
/fr/detail/villa/a-vendre/3080/tervuren/raw62354
/fr/detail/maison/a-vendre/8530/harelbeke/raw62256
/fr/detail/maison-de-maitre/a-vendre/9300/alost/raw61933
/fr/detail/maison/a-vendre/8610/handzame/raw60048
/fr/detail/maison/a-vendre/8680/koekelare/raw60045
/fr/detail/maison/a-vendre/8501/bissegem/raw60036
/fr/detail/appartement/a-vendre/8890/dadizele/raw60035
/fr/detail/maison/a-vendre/9700/audenarde/raw59956
/fr/detail/appartement/a-vendre/9600/renaix/raw59914
/fr/detail/maison/a-vendre/8630/houtem/raw59658
/fr/detail/maison/a-vendre/2310/rijkevorsel/raw

In [45]:
<h2 itemprop="name" title="Terrain  à vendre à Courcelles (VAR59104)" class="card-title text-ellipsis mb-1 d-inline with-small-icon land"><a href="/fr/detail/terrain-a-batir/a-vendre/6180/courcelles/var59104" title="Terrain à bâtir à vendre"><strong>Terrain à bâtir à vendre</strong></a></h2>

<h2 itemprop="name" title="Appartement  à vendre à La Hestre (VAR58606)" class="card-title text-ellipsis mb-1 d-inline with-small-icon appartment"><a href="/fr/detail/appartement/a-vendre/7170/la-hestre/var58606" title="Appartement à vendre"><strong>Appartement à vendre</strong></a></h2>

<h2 itemprop="name" title="Maison  à vendre à Ham-sur-Sambre (VAR57536)" class="card-title text-ellipsis mb-1 d-inline with-small-icon house"><a href="/fr/detail/maison/a-vendre/5190/ham-sur-sambre/var57536" title="Maison à vendre"><strong>Maison à vendre</strong></a></h2>

NameError: name 'html' is not defined

In [64]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains 
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup


url='https://immo.vlan.be/fr'
print('ready')
source_pages = []  # contains the html
source_pages_url = []
driver = webdriver.Chrome()
driver.implicitly_wait(10)
driver.get(url)
time.sleep(7)
 #try:
    ##elem = driver.find_element_by_xpath("//input[@placeholder='Où ? Ville, Code Postal, Province ou Région.']")
     #elemn = WebDriverWait(driver, 1000).until(
     #   EC.presence_of_all_elements_located((By.XPATH, "//input[@placeholder='Où ? Ville, Code Postal, Province ou Région.']")))
#except:
     #print('error')
elem = driver.find_element_by_xpath("//input[@placeholder='Où ? Ville, Code Postal, Province ou Région.']")
elem.clear()
elem.send_keys("8000")
elem.send_keys(Keys.RETURN)
time.sleep(7)
my_soup=BeautifulSoup(driver.page_source)
links = my_soup.find_all('a', attrs = {'rel':'nofollow'})
last = 0
for i in range(len(links)):
    if links[i].get('title') == 'suivant':
        last = int(links[i-1].get_text())
print(last)
for i in range(last):
    xpath_flat = "//h2[@class='card-title text-ellipsis mb-1 d-inline with-small-icon appartment']"
    xpath_house = "//h2[@class='card-title text-ellipsis mb-1 d-inline with-small-icon house']"
    elements_house = driver.find_elements_by_xpath(xpath_house)
    elements_flat = driver.find_elements_by_xpath(xpath_flat)
    
    elements = elements_flat + elements_house

    print(len(elements))
    for i in range(len(elements)):
        elem = elements[i]
        actions = ActionChains(driver)
        actions.move_to_element(elem)
        actions.key_down(Keys.LEFT_CONTROL)
        actions.click(elem)
        actions.perform()
        time.sleep(3)
        driver.switch_to_window(driver.window_handles[1])
        source_pages.append(driver.page_source)
        source_pages_url.append(driver.current_url)
        driver.close()
        add_to_csv(source_pages[-1])
        driver.switch_to_window(driver.window_handles[0])
    time.sleep(5)
    elem = driver.find_element_by_xpath("//i[@class='fa fa-angle-right']")
    actions = ActionChains(driver)
    actions.click(elem)
    actions.perform()
    time.sleep(5)
    elem = driver.find_element_by_xpath("//i[@class='fa fa-angle-right']")
    actions = ActionChains(driver)
    actions.click(elem)
    actions.perform()
    time.sleep(5)

driver.close()

ready
167
27
<ipython-input-64-e779408d4081>:55: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[1])
<ipython-input-64-e779408d4081>:60: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[0])


AttributeError: 'NoneType' object has no attribute 'find'

In [25]:
print(len(source_pages))
print(len(source_pages_url))
print(source_pages_url)

3
3
['https://immo.vlan.be/fr/detail/appartement/a-vendre/7170/la-hestre/var58606', 'https://immo.vlan.be/fr/detail/appartement/a-vendre/2260/tongerlo/raw65225', 'https://immo.vlan.be/fr/detail/appartement/a-vendre/8890/dadizele/raw60035']


In [15]:
limit = soup.find_all('a', attrs = {'rel':'nofollow'})
for i in range(len(limit)):
    if limit[i].get('title') == 'suivant':
        last = int(limit[i-1].get_text())
print(last)


167


In [50]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains 
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
from contextlib import contextmanager


url='https://immo.vlan.be/fr'
print('ready')
source_pages = []  # contains the html
source_pages_url = []
driver = webdriver.Chrome()
driver.implicitly_wait(10)
driver.get(url)
time.sleep(7)
elem = driver.find_element_by_xpath("//input[@placeholder='Où ? Ville, Code Postal, Province ou Région.']")
elem.clear()
elem.send_keys("8000")
elem.send_keys(Keys.RETURN)
time.sleep(7)
my_soup=BeautifulSoup(driver.page_source)
links = my_soup.find_all('a', attrs = {'rel':'nofollow'})
last = 0
for i in range(len(links)):
    if links[i].get('title') == 'suivant':
        last = int(links[i-1].get_text())
print(last)
last = 2 #to be removed
for i in range(last):
    xpath_flat = "//h2[@class='card-title text-ellipsis mb-1 d-inline with-small-icon appartment']"
    xpath_house = "//h2[@class='card-title text-ellipsis mb-1 d-inline with-small-icon house']"
    try:
        elements_flat = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, xpath_flat)))
        elements_house = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, xpath_house)))
        elements = elements_flat + elements_house
    except:
        print('could not find flat or house. Stop at page ', i)

    print(len(elements))
    for i in range(2): #range(len(elements)): te be change
        elem = elements[i]
        actions = ActionChains(driver)
        actions.move_to_element(elem)
        actions.key_down(Keys.LEFT_CONTROL)
        actions.click(elem)
        with wait_for_new_window(driver):
            actions.perform()
            driver.switch_to_window(driver.window_handles[1])
            source_pages.append(driver.page_source)
            source_pages_url.append(driver.current_url)
            driver.close()
        driver.switch_to_window(driver.window_handles[0])
    time.sleep(5)
    elem = driver.find_element_by_xpath("//i[@class='fa fa-angle-right']")
    actions = ActionChains(driver)
    actions.click(elem)
    actions.perform()
    time.sleep(5)
    elem = driver.find_element_by_xpath("//i[@class='fa fa-angle-right']")
    actions = ActionChains(driver)
    actions.click(elem)
    actions.perform()
    time.sleep(5)

driver.close()



ready
167
27
<ipython-input-50-84a7a0df9ac1>:55: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[1])


TimeoutException: Message: 


In [35]:
@contextmanager
def wait_for_new_window(driver, timeout=10):
    handles_before = driver.window_handles
    yield
    WebDriverWait(driver, timeout).until(
        lambda driver: len(handles_before) != len(driver.window_handles))

In [62]:
def add_to_csv(source_page):
    soup = BeautifulSoup(source_page, 'lxml')
    try:
        adid = soup.find("meta", {"name": "cXenseParse:rob-immo-property-adid"}).get("content")
    except:
        adid=""
    try:
        bedrooms = soup.find("meta", {"name": "cXenseParse:rob-immo-property-bedrooms"}).get("content")
    except:
        bedrooms=""
    try:
        city = soup.find("meta", {"name": "cXenseParse:rob-immo-property-city"}).get("content")
    except:
        city=""
    try:
        price = soup.find("meta", {"name": "cXenseParse:rob-immo-property-price"}).get("content")
    except:
        price=""
    try:
        transaction_type = soup.find("meta", {"name": "cXenseParse:rob-immo-transaction-type"}).get("content")
    except:
        transaction_type=""
    try:
        subtype = soup.find("meta", {"name": "cXenseParse:rob-immo-property-subtype"}).get("content")
    except:
        subtype=""
    try:
        zipcode = soup.find("meta", {"name": "cXenseParse:rob-immo-property-zipcode"}).get("content")
    except:
        zipcode=""
    try:    
        surface = soup.find("div", {"title": "Surface habitable"}).find("div", {"class":"fs-4"}).text
    except:
        surface = soup.find("div", {"title": "Surface habitable"}).find("div", {"class":"fs-4"}).text
        adid=""
    with open('house_data.csv', 'a') as file:
        string_to_add =adid+','+bedrooms+','+city+','+price+','+transaction_type+','+subtype+','+zipcode+','+surface+',,' ##removed adresses because error
        file.write(string_to_add+'\n')


In [58]:
with open('house_data.csv', 'a') as file:
    file.write("id,chambres, city,price,transaction_type,sous-type,code-postal,surface,,\n")

In [76]:

class ImmoWebScrapping:
    def __init__(self, url):
        self.driver = webdriver.Chrome()
        self.pages_to_scrap = []
        self.last = 0
        self.number_page_scrapped = 0
        self.driver.get(url)
        self.change_page = False
        time.sleep(10)
        self.driver.maximize_window()
        print('ready')

    def start_research(self, xpath, research_sentence, pushbutton=None):
        current_url = self.driver.current_url
        elem = self.driver.find_element_by_xpath(xpath)
        elem.clear()
        elem.send_keys(research_sentence)
        elem.send_keys(Keys.RETURN)
        time.sleep(3)
        elem.send_keys(Keys.RETURN)
        time.sleep(3)
        if pushbutton!= None:
            button = self.driver.find_element_by_xpath(pushbutton) 
            actions = ActionChains(self.driver)
            actions.move_to_element(elem)
            actions.key_down(Keys.LEFT_CONTROL)
            actions.click(elem)
            actions.perform()
        WebDriverWait(self.driver, 15).until(EC.url_changes(current_url))

            

    def get_number_pages(self, last_entry_location):
        my_soup = BeautifulSoup(self.driver.page_source, features="html.parser")
        links = my_soup.find_all(last_entry_location[0], last_entry_location[1])
        self.last = int(links[-1].get_text())

    def scrap_page(self, xpaths, next_page_xpath):
        for i in range(self.last):
            print('scrapping page ', i + 1)
            elements = []
            for xpath in xpaths:
                wait = WebDriverWait(self.driver, 60)
                elements_for_xpath = wait.until(EC.presence_of_all_elements_located((By.XPATH, xpath)))
                #elements_for_xpath = self.driver.find_elements_by_xpath(xpath)
                elements += elements_for_xpath
            print(len(elements), elements)
            for j in range(len(elements)):
                elem = elements[j]
                windows_before = self.driver.window_handles
                actions = ActionChains(self.driver)
                actions.move_to_element(elem)
                actions.key_down(Keys.LEFT_CONTROL)
                actions.click(elem)
                actions.perform()
                WebDriverWait(self.driver, 20).until(EC.new_window_is_opened(windows_before))
                self.driver.switch_to.window(self.driver.window_handles[1])
                source_page = self.driver.page_source
                self.driver.close()
                self.driver.switch_to.window(self.driver.window_handles[0])
                yield source_page
            self.next_page(next_page_xpath)
            #self.change_page=True
            #self.number_page_scrapped += 1

    def list_complete(self):
        return self.change_page

    def next_page(self, xpath):
        current_url = self.driver.current_url
        self.driver.maximize_window()
        wait = WebDriverWait(self.driver, 60)
        elem = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        actions = ActionChains(self.driver)
        actions.click(elem)
        actions.perform()
        time.sleep(10)
        #elem = self.driver.find_element_by_xpath(xpath)
        #actions = ActionChains(self.driver)
        #actions.click(elem)
        #actions.perform()
        WebDriverWait(self.driver, 15).until(EC.url_changes(current_url))
        self.change_page=False


    def research_completed(self):
        return self.number_page_scrapped == self.last

    def close(self):
        self.driver.quit()

In [77]:

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
from page_scrapping import b_soup_immo

#with open('data.csv', 'w') as file:
    #file.write("adid, bedrooms, city, price, transaction_type, subtype, "
    #           "zipcode, adresse, surface, type_prop, etat, facades, surface_terrain, garden, "
    #           "garden_surface, terrasse, terrasse_surface, piscine, nombre_chambres, garage, surface_habitable, "
    #           "meuble, salles_de_bain, toilettes, cuisine_equipee, cheminee\n")

my_scrapper = ImmoWebScrapping('https://www.immoweb.be/fr/recherche/maison-et-appartement/a-vendre?countries=BE&page=1&orderBy=relevance')
path_to_research_field = '//input[@placeholder="Entrez une ville ou un code postal"]'
button_xpath = '//button[@id="searchBoxSubmitButton"]'             #<button id="searchBoxSubmitButton" type="submit" class="button button--primary"><span class="button__label">Rechercher</span></button>
input_sentence = 'Mons'
#my_scrapper.start_research(path_to_research_field, input_sentence)
xpaths = ["//a[@class='card__title-link']"]
last_entry_location = ('span', {'class':'button__label'})
next_page_xpath =  "//a[@class='pagination__link pagination__link--next button button--text button--size-small']"
my_scrapper.get_number_pages(last_entry_location)
while True:
    for html_page in my_scrapper.scrap_page(xpaths,next_page_xpath):
        print('page scrapped')
        #with open('data.csv', 'a') as file:
        #    file.write(b_soup_immo(html_page)+"\n")
    


ready
scrapping page  1
30 [<selenium.webdriver.remote.webelement.WebElement (session="8acb79299538de9329b1a2a3d71195b5", element="eb6c1e1f-ae6f-4bb7-9496-06cad85e0fd1")>, <selenium.webdriver.remote.webelement.WebElement (session="8acb79299538de9329b1a2a3d71195b5", element="0a76ae4c-519b-4e64-bdca-dbb0d25b3def")>, <selenium.webdriver.remote.webelement.WebElement (session="8acb79299538de9329b1a2a3d71195b5", element="b500faf8-c6fe-4450-a640-e7d1b305f06e")>, <selenium.webdriver.remote.webelement.WebElement (session="8acb79299538de9329b1a2a3d71195b5", element="87e15b3f-51f7-48ec-9aca-0a4b9f1d65f5")>, <selenium.webdriver.remote.webelement.WebElement (session="8acb79299538de9329b1a2a3d71195b5", element="86a46ce1-9c78-4a27-b3dc-36661ead8157")>, <selenium.webdriver.remote.webelement.WebElement (session="8acb79299538de9329b1a2a3d71195b5", element="c05d37e9-fca4-4915-8600-3c44f5d582b3")>, <selenium.webdriver.remote.webelement.WebElement (session="8acb79299538de9329b1a2a3d71195b5", element="10652

WebDriverException: Message: chrome not reachable
  (Session info: chrome=90.0.4430.93)


In [ ]:
url = 'https://www.immoweb.be/fr/recherche/maison-et-appartement/a-vendre?countries=BE&page=1&orderBy=relevance'
driver = webdriver.Chrome()
driver.get(url)
time.sleep(10)
driver.maximize_window()
wait = WebDriverWait(driver, 60)
xpath = "//a[@class='pagination__link pagination__link--next button button--text button--size-small']"
next_page_xpath = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
print(next_page_xpath)
windows_before = driver.window_handles
actions = ActionChains(driver)
actions.move_to_element(next_page_xpath)
actions.key_down(Keys.LEFT_CONTROL)
actions.click(next_page_xpath)
actions.perform()
WebDriverWait(driver, 20).until(EC.new_window_is_opened(windows_before))
